In [403]:
#Importamos las librerias
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.3f}'.format
from math import acos, cos, sin, radians


In [7]:
#Cargamos las bases de datos
exa_barrios_cali = pd.read_csv('exa_barrios_cali.csv')
exa_dispositivos_cali = pd.read_csv('exa_dispositivos_cali.csv')
exa_trx_clientes = pd.read_csv('exa_trx_clientes.csv')

In [11]:
exa_barrios_cali.head()

,codigo,nombre
0,737998257,La Campina
1,737999435,Guayaquil
2,737998219,Vipasa
3,737996837,Lourdes
4,737997031,Villanueva


In [14]:
exa_dispositivos_cali.head()

,tipo,codigo,latitud,longitud,id_barrio
0,POS,1024702,3.451135,-76.530893,737998832
1,POS,1076402,3.446585,-76.517672,738000381
2,POS,1077002,3.446585,-76.517672,738000381
3,POS,3342404,3.476909,-76.485286,737998905
4,POS,3378003,3.485575,-76.516418,737998174


In [16]:
exa_trx_clientes.head()

,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,6861792605025121524,1,POS,14812028,3,22218042.0
1,1552970090928921643,1,POS,14388938,3,21583407.0
2,-4075904842556875666,1,POS,12370698,1,6185349.0
3,3539339627033221084,1,POS,10313583,2,10313583.0
4,5595554501572542610,1,POS,14358071,4,28716142.0


In [18]:
#Unimos las tablas usando las llaves primarias de las mismas
joined1 = pd.merge(exa_dispositivos_cali,exa_trx_clientes,left_on = 'codigo',right_on = 'cod_dispositivo')

In [19]:
joined1.head()

,tipo,codigo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,POS,10316933,3.443583,-76.521854,738000401,-584593722589695970,1,POS,10316933,1,5158466.5
1,POS,10316933,3.443583,-76.521854,738000401,-501888173821165749,1,POS,10316933,1,5158466.5
2,POS,10520575,3.463253,-76.530402,737998194,-8940569430134622363,1,POS,10520575,3,15780862.5
3,POS,10520575,3.463253,-76.530402,737998194,-844575220084151988,1,POS,10520575,1,5260287.5
4,POS,10520575,3.463253,-76.530402,737998194,1758144660880199993,1,POS,10520575,1,5260287.5


In [91]:
#Unimos con la otra tabla
tabla_unida = pd.merge(exa_barrios_cali,joined1,left_on = 'codigo' ,right_on = 'id_barrio')

In [92]:
tabla_unida = tabla_unida.rename(columns = {'codigo_x':'codigo_barrio','codigo_y':'codigo_dispositivo','nombre':'nombre_barrio'})

In [93]:
tabla_unida.head()

,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.479572,-76.525582,737998257,-2393994463924208659,1,POS,12202503,2,12202503.0
1,737998257,La Campina,POS,10521045,3.480517,-76.525275,737998257,-5675531380015678594,1,POS,10521045,1,5260522.5
2,737998257,La Campina,POS,10521045,3.480517,-76.525275,737998257,-5066077049535851626,1,POS,10521045,1,5260522.5
3,737998257,La Campina,POS,13238282,3.479491,-76.525818,737998257,-5791329717693361241,1,POS,13238282,1,6619141.0
4,737998257,La Campina,POS,12567517,3.480958,-76.525153,737998257,-397966594878551344,1,POS,12567517,1,6283758.5


In [174]:
#1 ¿Cuáles son los barrios en los cuales se realizaron los movimientos de al menos el 51% del dinero total tranzado
# por cada cliente? Considere que los clientes pueden tener más de un barrio.


In [426]:
#Agrupamos los datos
grouped = tabla_unida.groupby(['num_doc','nombre_barrio'],as_index = False)['mnt_total_trx'].sum()

In [427]:
grouped.head()


,num_doc,nombre_barrio,mnt_total_trx
0,-9218698523982046880,El Gran Limonar,1003163089.500
1,-9218698523982046880,Lili,538063140.500
2,-9218698523982046880,San Vicente,1656631596.000
3,-9218698523982046880,Sucre,538060458.000
4,-9218698523982046880,Unidad Residencial Santiago de Cali,552211796.500


In [237]:
#Creamos la variable porcentaje

suma_cliente = tabla_unida.groupby(['num_doc'], as_index = False)['mnt_total_trx'].sum()

In [238]:
suma_cliente

,num_doc,mnt_total_trx
0,-9218698523982046880,4840352147.000
1,-9214605575224426554,2828.000
2,-9211522780831135515,412.500
3,-9210956249921547631,5301203.500
4,-9210147727599394517,58456.000
...,...,...
9995,9219183828174672272,21811240.500
9996,9220077217609243452,5757648245.000
9997,9222313660278362882,257.000
9998,9222542349226241781,1104422232.000


In [239]:
#Unimos las tablas
tabla_con_sumas = pd.merge(grouped,suma_cliente,left_on = 'num_doc',right_on = 'num_doc')

In [240]:
tabla_con_sumas

,num_doc,nombre_barrio,mnt_total_trx_x,mnt_total_trx_y
0,-9218698523982046880,El Gran Limonar,1003163089.500,4840352147.000
1,-9218698523982046880,Lili,538063140.500,4840352147.000
2,-9218698523982046880,San Vicente,1656631596.000,4840352147.000
3,-9218698523982046880,Sucre,538060458.000,4840352147.000
4,-9218698523982046880,Unidad Residencial Santiago de Cali,552211796.500,4840352147.000
...,...,...,...,...
55761,9222313660278362882,El Sena,257.000,257.000
55762,9222542349226241781,El Sena,552208326.500,1104422232.000
55763,9222542349226241781,Santa Teresita,361.000,1104422232.000
55764,9222542349226241781,Unidad Residencial Santiago de Cali,552213544.500,1104422232.000


In [242]:
#Renombramos las columnas

tabla_con_sumas = tabla_con_sumas.rename(columns = {'mnt_total_trx_x':'mnt_total_trx_barrio_cliente','mnt_total_trx_y':'mnt_total_trx_total_cliente'})
tabla_con_sumas


,num_doc,nombre_barrio,mnt_total_trx_barrio_cliente,mnt_total_trx_total_cliente
0,-9218698523982046880,El Gran Limonar,1003163089.500,4840352147.000
1,-9218698523982046880,Lili,538063140.500,4840352147.000
2,-9218698523982046880,San Vicente,1656631596.000,4840352147.000
3,-9218698523982046880,Sucre,538060458.000,4840352147.000
4,-9218698523982046880,Unidad Residencial Santiago de Cali,552211796.500,4840352147.000
...,...,...,...,...
55761,9222313660278362882,El Sena,257.000,257.000
55762,9222542349226241781,El Sena,552208326.500,1104422232.000
55763,9222542349226241781,Santa Teresita,361.000,1104422232.000
55764,9222542349226241781,Unidad Residencial Santiago de Cali,552213544.500,1104422232.000


In [243]:
#Creando la variable porcentaje

porcentaje_trx = tabla_con_sumas['mnt_total_trx_barrio_cliente']/tabla_con_sumas['mnt_total_trx_total_cliente'] 

In [244]:
porcentaje_trx 

0       0.207
1       0.111
2       0.342
3       0.111
4       0.114
         ... 
55761   1.000
55762   0.500
55763   0.000
55764   0.500
55765   1.000
Length: 55766, dtype: float64

In [245]:
#Agregamos la columna porcentajes
tabla_con_sumas.insert(4,"Porcentaje",porcentaje_trx)

In [246]:
tabla_con_sumas

,num_doc,nombre_barrio,mnt_total_trx_barrio_cliente,mnt_total_trx_total_cliente,Porcentaje
0,-9218698523982046880,El Gran Limonar,1003163089.500,4840352147.000,0.207
1,-9218698523982046880,Lili,538063140.500,4840352147.000,0.111
2,-9218698523982046880,San Vicente,1656631596.000,4840352147.000,0.342
3,-9218698523982046880,Sucre,538060458.000,4840352147.000,0.111
4,-9218698523982046880,Unidad Residencial Santiago de Cali,552211796.500,4840352147.000,0.114
...,...,...,...,...,...
55761,9222313660278362882,El Sena,257.000,257.000,1.000
55762,9222542349226241781,El Sena,552208326.500,1104422232.000,0.500
55763,9222542349226241781,Santa Teresita,361.000,1104422232.000,0.000
55764,9222542349226241781,Unidad Residencial Santiago de Cali,552213544.500,1104422232.000,0.500


In [248]:
#Hallamos los valores de porcentaje mayores o iguales a 51%
tabla_mayor_a_51 = tabla_con_sumas[tabla_con_sumas.Porcentaje >= 0.51]

In [249]:
tabla_mayor_a_51

,num_doc,nombre_barrio,mnt_total_trx_barrio_cliente,mnt_total_trx_total_cliente,Porcentaje
8,-9214605575224426554,Urbanización Militar,2828.000,2828.000,1.000
9,-9211522780831135515,San Vicente,412.500,412.500,1.000
10,-9210956249921547631,San Fernando Nuevo,5301203.500,5301203.500,1.000
18,-9204167197570414556,Urbanización San Juaquín,1121218306.000,1121218306.000,1.000
35,-9201713395891677864,Versalles,2761047222.500,2784478432.500,0.992
...,...,...,...,...,...
55733,9219111613418646473,Chapinero,1663347048.000,2609796495.500,0.637
55747,9219183828174672272,San Vicente,21811240.500,21811240.500,1.000
55757,9220077217609243452,Siete de Agosto,5191647216.000,5757648245.000,0.902
55761,9222313660278362882,El Sena,257.000,257.000,1.000


In [250]:
# Dejamos valores de barrio unicos
unico = pd.unique(tabla_mayor_a_51['nombre_barrio'])

In [251]:
#Barrios UNICOS con un porcentaje de transaccion por cliente mayor a 51%
unico

array(['Urbanización Militar', 'San Vicente', 'San Fernando Nuevo',
       'Urbanización San Juaquín', 'Versalles', 'Colseguros Andes',
       'Ciudad 2000', 'Calima', 'Olaya Herrera', 'Los Alcázares',
       'Chapinero', 'Unicentro Cali', 'Junin', 'Chipichape',
       'Ciudad Capri', 'Lili', 'San Fernando Viejo', 'La Merced',
       'La Rivera', 'Urbanización Río Lili', 'El Cedro', 'Santander',
       'Santa Rita', 'Primero de Mayo', 'Prados del Norte',
       'San Cayetano', 'El Sena', 'Bolivariano',
       'Á\x81rea Libre-Parque del Amor', 'Belén', 'El Gran Limonar',
       'Urbanización La Flora', 'Centenario', 'El Calvario',
       'Santa Mónica Residencial', 'Laureano Gómez', 'San Luis II',
       'Santa Teresita', 'Urbanización Ciudad Jardín', 'San Nicolás',
       'Ciudad Córdoba', 'Nueva Floresta del Ict',
       'Urbanización Tequendama', 'San Pedro', 'Juanambu',
       'Los Guaduales', 'La Flora', 'Mariano Ramos', 'El Diamante',
       'El Peñón', 'Julio Rincón', 'Cataya-El 

In [252]:
#2 ¿Cuáles son los dispositivos con transacciones de al menos 100 clientes diferentes?


In [254]:
tabla_unida

,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.480,-76.526,737998257,-2393994463924208659,1,POS,12202503,2,12202503.000
1,737998257,La Campina,POS,10521045,3.481,-76.525,737998257,-5675531380015678594,1,POS,10521045,1,5260522.500
2,737998257,La Campina,POS,10521045,3.481,-76.525,737998257,-5066077049535851626,1,POS,10521045,1,5260522.500
3,737998257,La Campina,POS,13238282,3.479,-76.526,737998257,-5791329717693361241,1,POS,13238282,1,6619141.000
4,737998257,La Campina,POS,12567517,3.481,-76.525,737998257,-397966594878551344,1,POS,12567517,1,6283758.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94605,737998035,Ciudadela del Río,POS,11044369,3.425,-76.465,737998035,-3765831630548519698,1,POS,11044369,1,5522184.500
94606,737998035,Ciudadela del Río,POS,11044369,3.425,-76.465,737998035,2580778409722080163,1,POS,11044369,39,215365195.500
94607,737998035,Ciudadela del Río,POS,11044369,3.425,-76.465,737998035,-6052852033626412092,1,POS,11044369,1,5522184.500
94608,737998035,Ciudadela del Río,POS,11044369,3.425,-76.465,737998035,8994524193252232475,1,POS,11044369,2,11044369.000


In [306]:
# Se realiza la agrupacion de datos
dispositivos_clientes = tabla_unida.groupby(['codigo_dispositivo','num_doc'],as_index = False)['num_trx'].count()

In [308]:
dispositivos_clientes

,codigo_dispositivo,num_doc,num_trx
0,60,-9098139820114839305,1
1,60,-8967040706319190407,1
2,60,-8964890812302625768,1
3,60,-8854286616571007558,1
4,60,-8804468740174950342,1
...,...,...,...
91364,2100027645,1466006045077319816,1
91365,2100027645,2805367884045523793,1
91366,2100027645,5225504286068046615,1
91367,2100027645,7839402912448145784,1


In [323]:
#Quitamos duplicados de los clientes
dispositivos_clientes_sin_duplicados = dispositivos_clientes.drop_duplicates(subset = ['num_doc'])
dispositivos_clientes_sin_duplicados

,codigo_dispositivo,num_doc,num_trx
0,60,-9098139820114839305,1
1,60,-8967040706319190407,1
2,60,-8964890812302625768,1
3,60,-8854286616571007558,1
4,60,-8804468740174950342,1
...,...,...,...
91182,1990625730,-1623831492365706138,1
91263,2033226660,-7891689009115340241,1
91274,2033226660,-1145599271250714953,1
91308,2047126840,5866388254324037176,1


In [324]:
#Se agrupan los datos contando los clientes unicos por dispositivo

dispositivos_cantidad_clientes = dispositivos_clientes_sin_duplicados.groupby(
    ['codigo_dispositivo'],as_index = False)['num_doc'].count()
dispositivos_cantidad_clientes

,codigo_dispositivo,num_doc
0,60,346
1,61,177
2,62,119
3,64,158
4,75,45
...,...,...
800,1966725143,1
801,1990625730,2
802,2033226660,2
803,2047126840,1


In [328]:
#Hallando los dispositivos con 100 o mas clientes individuales

dispositivos_con_mas_de_100_clientes_diferentes = dispositivos_cantidad_clientes[dispositivos_cantidad_clientes.num_doc >= 100]
dispositivos_con_mas_de_100_clientes_diferentes

,codigo_dispositivo,num_doc
0,60,346
1,61,177
2,62,119
3,64,158
5,77,158
7,205,137
8,265,260
11,514,214
12,600,127
14,605,230


In [329]:
#3 ¿Cuáles son los 5 barrios donde la mayor cantidad de clientes únicos realizan transacciones en dispositivos tipo POS? La respuesta debe incluir la cantidad de clientes asociados a estos barrios.

In [332]:
#Quitamos duplicados de clientes
tabla_unida_sin_clientes_duplicados  = tabla_unida.drop_duplicates(subset = ['num_doc'])
tabla_unida_sin_clientes_duplicados 


,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.480,-76.526,737998257,-2393994463924208659,1,POS,12202503,2,12202503.000
1,737998257,La Campina,POS,10521045,3.481,-76.525,737998257,-5675531380015678594,1,POS,10521045,1,5260522.500
2,737998257,La Campina,POS,10521045,3.481,-76.525,737998257,-5066077049535851626,1,POS,10521045,1,5260522.500
3,737998257,La Campina,POS,13238282,3.479,-76.526,737998257,-5791329717693361241,1,POS,13238282,1,6619141.000
4,737998257,La Campina,POS,12567517,3.481,-76.525,737998257,-397966594878551344,1,POS,12567517,1,6283758.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94287,737998685,El Peñón,POS,13283411,3.450,-76.541,737998685,7376278692136269320,1,POS,13283411,1,6641705.500
94454,737997007,Alfonso López II,POS,13388665,3.458,-76.480,737997007,3630456067817960412,1,POS,13388665,4,26777330.000
94461,737997007,Alfonso López II,POS,13388665,3.458,-76.480,737997007,-8796016162006914712,1,POS,13388665,1,6694332.500
94545,738000408,Prados del Limonar,DISPENSADOR,3094,3.392,-76.525,738000408,-5337620632397778396,1,DISPENSADOR,3094,2,3094.000


In [337]:
#Filtrando por dispositivo tipo POS 
dispositivos_POS = tabla_unida_sin_clientes_duplicados[tabla_unida_sin_clientes_duplicados.tipo == 'POS']
dispositivos_POS

,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
0,737998257,La Campina,POS,12202503,3.480,-76.526,737998257,-2393994463924208659,1,POS,12202503,2,12202503.000
1,737998257,La Campina,POS,10521045,3.481,-76.525,737998257,-5675531380015678594,1,POS,10521045,1,5260522.500
2,737998257,La Campina,POS,10521045,3.481,-76.525,737998257,-5066077049535851626,1,POS,10521045,1,5260522.500
3,737998257,La Campina,POS,13238282,3.479,-76.526,737998257,-5791329717693361241,1,POS,13238282,1,6619141.000
4,737998257,La Campina,POS,12567517,3.481,-76.525,737998257,-397966594878551344,1,POS,12567517,1,6283758.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94132,737998685,El Peñón,POS,14615447,3.449,-76.540,737998685,4834442546807716495,1,POS,14615447,1,7307723.500
94229,737998685,El Peñón,POS,15584139,3.451,-76.538,737998685,-3768157435786418819,1,POS,15584139,1,7792069.500
94287,737998685,El Peñón,POS,13283411,3.450,-76.541,737998685,7376278692136269320,1,POS,13283411,1,6641705.500
94454,737997007,Alfonso López II,POS,13388665,3.458,-76.480,737997007,3630456067817960412,1,POS,13388665,4,26777330.000


In [340]:
#Agrupando clientes por barrio

clientes_barrio_POS = dispositivos_POS.groupby(['nombre_barrio'],as_index = False)['num_doc'].count()
clientes_barrio_POS

,nombre_barrio,num_doc
0,12 de Octubre,1
1,3 de Julio,5
2,Acueducto San Antonio,5
3,Aguablanca,1
4,Alameda,96
...,...,...
153,Villa del Prado-El Guabito,4
154,Villa del Sol,4
155,Villanueva,2
156,Vipasa,108


In [345]:
#Ordenamos la tabla de mayor a menor en funcion del numero de clientes diferentes
clientes_barrio_POS_ordenado = clientes_barrio_POS.sort_values('num_doc',ascending = False)
clientes_barrio_POS_ordenado

,nombre_barrio,num_doc
126,San Vicente,269
144,Urbanización Ciudad Jardín,237
48,El Gran Limonar,185
156,Vipasa,108
59,El Sena,106
...,...,...
62,Fepicol,1
58,El Retiro,1
104,Normandía,1
134,Sector Meléndez,1


In [347]:
# Cinco primeros barrios
clientes_barrio_POS_ordenado.head(5)

,nombre_barrio,num_doc
126,San Vicente,269
144,Urbanización Ciudad Jardín,237
48,El Gran Limonar,185
156,Vipasa,108
59,El Sena,106


In [ ]:
#4 ¿Cuáles son las 10 distancias únicas (en kilómetros) de los dispositivos más alejados entre sí del barrio Sucre?

In [371]:
#Filtramos los datos por barrio: Sucre
tabla_unida_sucre = tabla_unida[tabla_unida.nombre_barrio == 'Sucre']
tabla_unida_sucre

,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
6940,737999416,Sucre,POS,10551810,3.450,-76.528,737999416,2482768808625848045,1,POS,10551810,1,5275905.000
6941,737999416,Sucre,POS,11961257,3.444,-76.522,737999416,1638947603092353476,1,POS,11961257,1,5980628.500
6942,737999416,Sucre,POS,11961257,3.444,-76.522,737999416,8995680267260645346,1,POS,11961257,1,5980628.500
6943,737999416,Sucre,POS,13788534,3.450,-76.528,737999416,-2617888623937358049,1,POS,13788534,1,6894267.000
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7749,737999416,Sucre,POS,15432867,3.449,-76.526,737999416,-2559927509367529475,1,POS,15432867,1,7716433.500
7750,737999416,Sucre,POS,15432867,3.449,-76.526,737999416,-4720990395795836959,1,POS,15432867,1,7716433.500
7751,737999416,Sucre,POS,15432867,3.449,-76.526,737999416,-6806935765734096567,3,POS,15432867,1,7716433.500
7752,737999416,Sucre,POS,14717417,3.443,-76.526,737999416,-5898996727764225756,1,POS,14717417,1,7358708.500


In [373]:
#Quitamos dispositivos repetidos para dejarlos Únicos 
datos_dispositivos_unicos = tabla_unida_sucre.drop_duplicates(subset = ['codigo_dispositivo'])
datos_dispositivos_unicos


,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx
6940,737999416,Sucre,POS,10551810,3.450,-76.528,737999416,2482768808625848045,1,POS,10551810,1,5275905.000
6941,737999416,Sucre,POS,11961257,3.444,-76.522,737999416,1638947603092353476,1,POS,11961257,1,5980628.500
6943,737999416,Sucre,POS,13788534,3.450,-76.528,737999416,-2617888623937358049,1,POS,13788534,1,6894267.000
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500
6945,737999416,Sucre,POS,10466894,3.444,-76.523,737999416,-8251766315295553449,1,POS,10466894,1,5233447.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7742,737999416,Sucre,POS,15042526,3.450,-76.527,737999416,-4482800304130342256,1,POS,15042526,2,15042526.000
7744,737999416,Sucre,POS,14086565,3.451,-76.526,737999416,7497025041483308553,1,POS,14086565,1,7043282.500
7746,737999416,Sucre,POS,14293500,3.444,-76.522,737999416,-8804792444315738258,1,POS,14293500,1,7146750.000
7747,737999416,Sucre,POS,15432867,3.449,-76.526,737999416,-6806935765734096567,2,POS,15432867,1,7716433.500


In [395]:
# Se usa la formula del Semiverseno y se crea una funcion con base en ella
    
def haversine(lon1, lat1, lon2, lat2):
    
    
    lon1 = np.radians(lon1)
    lat1 = np.radians(lat1)
    lon2 = np.radians(lon2)
    lat2 = np.radians(lat2)

    r = 6371
    
    dlon = np.subtract(lon2, lon1)
    dlat = np.subtract(lat2, lat1)

    a = np.add(np.power(np.sin(np.divide(dlat,2)),2),
               np.multiply(np.cos(lat1),
                           np.multiply(np.cos(lat2),
                                       np.power(np.sin(np.divide(dlon, 2)), 2))
                           )
              )
    c = np.multiply(2, np.arcsin(np.sqrt(a)))

    return c*r

In [415]:
#Realizamos la iteracion por codigo de dispositivo

df_distancias = pd.DataFrame([])
for codigo in datos_dispositivos_unicos['codigo_dispositivo']:
    print(codigo)
    
    aux = datos_dispositivos_unicos[datos_dispositivos_unicos['codigo_dispositivo']!=codigo]
   
    row = datos_dispositivos_unicos[datos_dispositivos_unicos['codigo_dispositivo']==codigo]
   
    aux['codigo_consultado'] = codigo
    
    aux['distancia'] = aux.apply(lambda fila: haversine(row.longitud.iloc[0],
                                                    row.latitud.iloc[0],
                                                    fila['longitud'],
                                                    fila['latitud']),
                             axis=1)
    
    df_distancias = df_distancias.append(aux)
    


10551810
11961257
13788534
13648241
10466894
13393152
14281349
10338358
13038500
1076120916
1944124867
815
4648
4649
10337087
10936607
11967114
11971405
10319705
10304707
10305498
14784110
10805786
12514055
10907566
11820784
11170107
11186905
11111788
12165353
11833530
11260866
11856051
11940632
12392460
11432259
12154795
12176988
12379913
12402418
11419637
12752861
12383204
12703179
12649133
12441259
12732111
12608964
12697447
15611296
14132617
12812996
13463401
13047287
13215983
13557327
12245528
13335732
13331202
12998001
14911960
10526598
13539580
13554795
13758685
13509625
13928973
12356523
14509988
14348569
14140743
14413363
13348776
14987671
14727325
14476386
15040314
12550489
12549804
15098957
14038541
15216260
14839096
13565478
13594858
14614937
14667752
13135330
13076112
13130323
15608664
13145198
14050801
14979280
15197445
13598511
15369796
14025779
15042526
14086565
14293500
15432867
14717417


In [416]:
# Generamos la tabla con los datos de distancia y codigo consultado
df_distancias

,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx,codigo_consultado,distancia
6941,737999416,Sucre,POS,11961257,3.444,-76.522,737999416,1638947603092353476,1,POS,11961257,1,5980628.500,10551810,0.907
6943,737999416,Sucre,POS,13788534,3.450,-76.528,737999416,-2617888623937358049,1,POS,13788534,1,6894267.000,10551810,0.024
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,10551810,0.950
6945,737999416,Sucre,POS,10466894,3.444,-76.523,737999416,-8251766315295553449,1,POS,10466894,1,5233447.000,10551810,0.870
6946,737999416,Sucre,POS,13393152,3.450,-76.527,737999416,-6897316410614000589,1,POS,13393152,1,6696576.000,10551810,0.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7740,737999416,Sucre,POS,14025779,3.449,-76.527,737999416,-8571211501004469818,1,POS,14025779,1,7012889.500,14717417,0.709
7742,737999416,Sucre,POS,15042526,3.450,-76.527,737999416,-4482800304130342256,1,POS,15042526,2,15042526.000,14717417,0.862
7744,737999416,Sucre,POS,14086565,3.451,-76.526,737999416,7497025041483308553,1,POS,14086565,1,7043282.500,14717417,0.886
7746,737999416,Sucre,POS,14293500,3.444,-76.522,737999416,-8804792444315738258,1,POS,14293500,1,7146750.000,14717417,0.478


In [419]:
#Ordenamos de manera ascendente por la variable distancia

distancias_ordenadas = df_distancias.sort_values('distancia',ascending = False)
distancias_ordenadas


,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx,codigo_consultado,distancia
7651,737999416,Sucre,POS,13557327,3.450,-76.528,737999416,8022315577142909432,1,POS,13557327,1,6778663.500,13648241,0.992
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,13557327,0.992
7520,737999416,Sucre,POS,10305498,3.450,-76.528,737999416,-7880095217661455071,1,POS,10305498,1,5152749.000,13648241,0.989
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,10305498,0.989
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,10304707,0.985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7656,737999416,Sucre,POS,13331202,3.449,-76.528,737999416,2483632734918501545,1,POS,13331202,1,6665601.000,14140743,0.000
7116,737999416,Sucre,SAI,815,3.444,-76.523,737999416,-3714935560124678639,1,SAI,815,2,815.000,4648,0.000
7698,737999416,Sucre,POS,14839096,3.451,-76.526,737999416,1662208121665838208,1,POS,14839096,1,7419548.000,12998001,0.000
7362,737999416,Sucre,DISPENSADOR,4649,3.444,-76.523,737999416,1790293122065626101,1,DISPENSADOR,4649,3,6973.500,815,0.000


In [424]:
#Obtenemos los 10 datos mas grandes en distancia
distancias_ordenadas.head(10)

,codigo_barrio,nombre_barrio,tipo,codigo_dispositivo,latitud,longitud,id_barrio,num_doc,tipo_doc,canal,cod_dispositivo,num_trx,mnt_total_trx,codigo_consultado,distancia
7651,737999416,Sucre,POS,13557327,3.450,-76.528,737999416,8022315577142909432,1,POS,13557327,1,6778663.500,13648241,0.992
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,13557327,0.992
7520,737999416,Sucre,POS,10305498,3.450,-76.528,737999416,-7880095217661455071,1,POS,10305498,1,5152749.000,13648241,0.989
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,10305498,0.989
6944,737999416,Sucre,POS,13648241,3.444,-76.521,737999416,-8251766315295553449,1,POS,13648241,1,6824120.500,10304707,0.985
7517,737999416,Sucre,POS,10304707,3.450,-76.528,737999416,8479430282877258496,1,POS,10304707,1,5152353.500,13648241,0.985
7694,737999416,Sucre,POS,14038541,3.444,-76.522,737999416,8777604891246326148,1,POS,14038541,1,7019270.500,13557327,0.982
7651,737999416,Sucre,POS,13557327,3.450,-76.528,737999416,8022315577142909432,1,POS,13557327,1,6778663.500,14038541,0.982
7520,737999416,Sucre,POS,10305498,3.450,-76.528,737999416,-7880095217661455071,1,POS,10305498,1,5152749.000,14038541,0.979
7694,737999416,Sucre,POS,14038541,3.444,-76.522,737999416,8777604891246326148,1,POS,14038541,1,7019270.500,10305498,0.979


In [425]:
#Obtenemos las 10 distancias mas grandes en KM
distancias_ordenadas['distancia'].head(10)

7651   0.992
6944   0.992
7520   0.989
6944   0.989
6944   0.985
7517   0.985
7694   0.982
7651   0.982
7520   0.979
7694   0.979
Name: distancia, dtype: float64